In [1]:
from google.colab import drive                
drive.mount('/content/drive')   

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization, Add, Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, TensorBoard
from tensorflow.keras.regularizers import l2

from tensorflow.keras.utils import plot_model
from tensorflow.python.keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
import time
import pickle
import os

In [ ]:
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_start_time)

In [ ]:
class ResNet:
    def __init__(self, n, framework, channels_first=False, initial_lr=0.01, nb_epochs=100):
        self.n = n
        self.framework = framework
        self.initial_lr = initial_lr
        self.nb_epochs = nb_epochs
        self.weight_decay = 0.0005
        self.channels_first = channels_first
        self.data_format = "channels_first" if channels_first else "channels_last"
        self.bn_axis = 1 if channels_first else -1
        # Make model
        self.model = self.make_model()
        plot_model(self.model, to_file='ResNet-26L-100E.png')
#         SVG(model_to_dot(self.model).create(prog='dot', format='svg'))

    def subsumpling(self, output_channels, input_tensor):
        return Conv2D(output_channels, kernel_size=1, strides=(2,2), data_format=self.data_format, kernel_regularizer=l2(self.weight_decay))(input_tensor)

    def block(self, channles, input_tensor):

        shortcut = input_tensor
        x = BatchNormalization(axis=self.bn_axis)(input_tensor)
        x = Activation("relu")(x)
        x = Conv2D(channles, kernel_size=3, padding="same", data_format=self.data_format, kernel_regularizer=l2(self.weight_decay))(x)
        x = BatchNormalization(axis=self.bn_axis)(x)
        x = Activation("relu")(x)
        x = Conv2D(channles, kernel_size=3, padding="same", data_format=self.data_format, kernel_regularizer=l2(self.weight_decay))(x)
        return Add()([x, shortcut])

    def make_model(self):
        input = Input(shape=(3, 200, 200)) if self.channels_first else Input(shape=(200, 200, 3))
        x = Conv2D(16, kernel_size=3, padding="same", data_format=self.data_format, kernel_regularizer=l2(self.weight_decay))(input)
        for i in range(self.n):
            x = self.block(16, x)
        # 16x16x32
        x = self.subsumpling(32, x)
        for i in range(self.n):
            x = self.block(32, x)
        # 8x8x64
        x = self.subsumpling(64, x)
        for i in range(self.n):
            x = self.block(64, x)
        # Global Average Pooling
        x = GlobalAveragePooling2D(data_format=self.data_format)(x)
        x = Dense(4, activation="softmax")(x)
        # model
        model = Model(input, x)
        return model

    def lr_schduler(self, epoch):
        x = self.initial_lr
        if epoch >= self.nb_epochs * 0.5: x /= 10.0
        if epoch >= self.nb_epochs * 0.75: x /= 10.0
        return x

    def train(self, TRAIN_PATH, VALIDATION_PATH):
        self.model.summary()
        self.model.compile(optimizer=SGD(lr=self.initial_lr, momentum=0.9), loss="categorical_crossentropy", metrics=["acc"])
        traingen = ImageDataGenerator(
            rescale=1./255,
            width_shift_range=4./32,
            height_shift_range=4./32,
            horizontal_flip=True)
        valgen = ImageDataGenerator(
            rescale=1./255)
        # Callback
        time_cb = TimeHistory()
        lr_cb = LearningRateScheduler(self.lr_schduler)
        tensorboard = TensorBoard(log_dir="./logsRN-26L-100E/{}".format(time.time()), 
                          histogram_freq=0, 
                          write_graph=True, 
                          write_grads=False, 
                          write_images=False, 
                          embeddings_freq=0, 
                          embeddings_layer_names=None, 
                          embeddings_metadata=None, 
                          embeddings_data=None, 
                          update_freq='epoch')

        # Train
        history = self.model.fit_generator(traingen.flow_from_directory(TRAIN_PATH, 
                                                                        target_size=(200,200),
                                                                        class_mode='categorical'), 
                                           epochs=100,
                                           callbacks=[time_cb, lr_cb, tensorboard],
                                           validation_data = valgen.flow_from_directory( VALIDATION_PATH,
                                                                                          target_size=(200,200),
                                                                                          class_mode='categorical')).history
        history["time"] = time_cb.times
        
        
        self.model.save("cotton_disease-resnet-26L-100E.h5")

In [5]:

# Main function
def main(n, framework):
    # layers = 6n+2
    net = ResNet(n, framework, nb_epochs=1)
    base_dir = '/content/drive/My Drive/B.E PROJECT/CNN CROP/data/disease/input'

    TRAIN_PATH = os.path.join(base_dir, 'Training')
    VALIDATION_PATH = os.path.join(base_dir, 'Validation')
    net.train(TRAIN_PATH, VALIDATION_PATH)

if __name__ == "__main__":
    main(4, "keras_tf")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 200, 200, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 200, 200, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 200, 200, 16) 0           batch_normalization[0][0]        
_____________

In [ ]:
  !ls 

 cotton_disease-resnet50E44L.h5  'logsRN-3(100E 44L)'   sample_data
 drive				  ResNet.png


In [7]:
!zip -r "/content/resnet26L100E.zip" "/content/logsRN-26L-100E/"
from google.colab import files
files.download("/content/resnet26L100E.zip")

updating: content/logsRN-26L-100E/ (stored 0%)
updating: content/logsRN-26L-100E/1572362515.5789483/ (stored 0%)
updating: content/logsRN-26L-100E/1572362515.5789483/plugins/ (stored 0%)
updating: content/logsRN-26L-100E/1572362515.5789483/plugins/profile/ (stored 0%)
updating: content/logsRN-26L-100E/1572362515.5789483/plugins/profile/2019-10-29_15-22-43/ (stored 0%)
updating: content/logsRN-26L-100E/1572362515.5789483/plugins/profile/2019-10-29_15-22-43/local.trace (deflated 96%)
updating: content/logsRN-26L-100E/1572362515.5789483/events.out.tfevents.1572362518.34f4422ef852 (deflated 92%)
updating: content/logsRN-26L-100E/1572362515.5789483/events.out.tfevents.1572362563.34f4422ef852.profile-empty (deflated 5%)
